In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
df = pd.read_csv('训练集.csv')

In [25]:
df

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,package,promotion_last_5years,division,salary,satisfaction_level
0,13697,0.99,3,161.39,2,0,a,0,accounting,medium,0.681
1,1142,1.00,5,226.22,6,0,b,0,marketing,low,0.876
2,7954,0.91,5,199.45,2,0,e,0,sales,medium,0.955
3,2225,0.51,3,235.14,3,0,c,0,sales,low,0.817
4,9753,0.89,3,219.91,2,0,a,0,technical,low,0.920
...,...,...,...,...,...,...,...,...,...,...,...
11994,11971,0.89,4,150.45,3,0,e,0,accounting,high,0.849
11995,14966,0.81,5,221.28,5,0,b,0,sales,low,0.836
11996,7491,0.60,4,136.36,4,1,c,0,sales,medium,0.915
11997,12680,0.84,6,268.54,4,0,b,0,IT,low,0.099


In [26]:
df['salary'].value_counts()

low       5886
medium    5142
high       971
Name: salary, dtype: int64

In [27]:
mapping = {'low':1,'medium':2,'high':3}

In [28]:
df = df.replace({'salary':mapping})

In [29]:
for col in df.columns:
    if sum((df[col]).isnull())!= 0:
        print(col)

In [30]:
X_train = df.drop(columns=['id','satisfaction_level'])

In [31]:
y_train = df['satisfaction_level']

In [32]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
X_test = pd.read_csv('测试集.csv')

In [34]:
X_test = X_test.replace({'salary':mapping})

In [35]:
ind = X_test['id']

In [36]:
X_test = X_test.drop(columns=['id'])

In [37]:
from sklearn.pipeline import Pipeline
from feature_engine import categorical_encoders as ce
from sklearn.preprocessing import StandardScaler

In [38]:
prepipe = Pipeline([
                    ('onehot', ce.OneHotCategoricalEncoder(variables=['package','division'],drop_last=True)),
                    ('scale', StandardScaler())
])

In [39]:
prepipe.fit(X_train,y_train)

Pipeline(steps=[('onehot',
                 OneHotCategoricalEncoder(drop_last=True,
                                          variables=['package', 'division'])),
                ('scale', StandardScaler())])

In [40]:
X_train = prepipe.transform(X_train)

In [41]:
X_test = prepipe.transform(X_test)

In [42]:
outlier = ((X_train>3)|(X_train<-3)).any(axis=1)

In [43]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.001)

ridge.fit(X_train, y_train)

Ridge(alpha=0.001)

In [44]:
y_pre = ridge.predict(X_test)

In [45]:
y_pre

array([0.70193431, 0.37198223, 0.62720923, ..., 0.67031903, 0.66849728,
       0.44307824])

In [46]:
out = pd.DataFrame({'id':ind,'satisfaction_level':y_pre}).to_csv('submission1.csv',index=False)